In [1]:
import pandas as pd
df = pd.read_csv('../datasets/anos/2023/INMET_SE_SP_A726_PIRACICABA_01-01-2023_A_31-12-2023.CSV',header=8, sep=';',encoding_errors='ignore' )
df

,Data,Hora UTC,"PRECIPITAO TOTAL, HORRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m),"TEMPERATURA DO AR - BULBO SECO, HORARIA (C)",TEMPERATURA DO PONTO DE ORVALHO (C),TEMPERATURA MXIMA NA HORA ANT. (AUT) (C),TEMPERATURA MNIMA NA HORA ANT. (AUT) (C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREO HORARIA (gr) ( (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2023/01/01,0000 UTC,",2","950,8","951,1","950,5",NaN,"21,5","20,6","22,1",21,"21,7",20,100.0,89.0,94.0,67.0,"6,3","2,3",NaN
1,2023/01/01,0100 UTC,0,"951,1","951,1","950,7",NaN,"21,1","20,9","21,7","21,1","20,9","20,5",99.0,93.0,99.0,58.0,"5,2",",9",NaN
2,2023/01/01,0200 UTC,0,"951,1","951,3","951,1",NaN,21,"20,9","21,2","20,9","21,1","20,8",100.0,99.0,99.0,352.0,"2,2",",1",NaN
3,2023/01/01,0300 UTC,0,"950,6","951,1","950,5",NaN,"21,2",21,"21,3","20,9","21,1","20,8",100.0,98.0,99.0,151.0,"2,8",2,NaN
4,2023/01/01,0400 UTC,0,"949,7","950,6","949,7",NaN,"20,6","20,4","21,3","20,5",21,"20,3",99.0,96.0,99.0,87.0,"2,8","1,3",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023/12/31,1900 UTC,0,"946,8","947,1","946,8","1811,9","30,9","13,5","32,8","30,5","16,1","13,2",39.0,34.0,35.0,187.0,"5,8","2,8",NaN
8756,2023/12/31,2000 UTC,0,"946,6","946,8","946,6","1237,9","29,9","14,5","32,3","29,8","14,7","12,8",40.0,32.0,39.0,181.0,7,"2,6",NaN
8757,2023/12/31,2100 UTC,0,"947,4","947,4","946,6","682,7","27,6","16,3","30,4","27,6","16,3","14,8",50.0,39.0,50.0,160.0,"11,4","5,7",NaN
8758,2023/12/31,2200 UTC,0,"948,1","948,2","947,3","85,8","25,5","16,6","27,7","25,5","16,6","16,3",58.0,50.0,58.0,145.0,"11,4","7,2",NaN


### Processamentos que precisam ser feitos:
- Arrumar o nome das colunas: colocar o nome em minusculo, sem acento, sem caracteres especiais, sem espaço e objetivo.
- Arrumar colunas numéricas: Garantir que o tipo da coluna é o mesmo, otimizar o espaço consumido pela coluna. (por ex: Float32 consome menos espaço que Float64)
- Arrumar colunas de data e hora. O python tem um tipo "datetime" em que podemos fazer operações com horário, arrumar a coluna para esse formato.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8760 non-null   object 
 1   Hora UTC                                               8760 non-null   object 
 2   PRECIPITAO TOTAL, HORRIO (mm)                          8759 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8759 non-null   object 
 4   PRESSO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)         8759 non-null   object 
 5   PRESSO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)        8759 non-null   object 
 6   RADIACAO GLOBAL (Kj/m)                                 4727 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (C)            8759 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (C)             

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from unidecode import unidecode

def transformar_dataframe(df):
    """
    Aplica as transformações no DataFrame.

    Args:
        df: DataFrame Pandas.

    Returns:
        DataFrame: DataFrame transformado.
    """

    df = df.dropna(axis=1, how='all').fillna(0)
    # Normalizar nomes de colunas
    for col in df.columns:
        df.rename(columns={col: unidecode(col), }, inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)

    # Converter colunas de data e hora
    df['data_hora'] = pd.to_datetime(df['data'] + ' ' + df['hora_utc'], format='%Y/%m/%d %H%M UTC')
    df.drop(['data', 'hora_utc'], axis=1, inplace=True)

    colunas_numericas = ['precipitacao_total_horario_mm',
       'pressao_atmosferica_ao_nivel_da_estacao_horaria_mb',
       'pressao_atmosferica_maxna_hora_ant_aut_mb',
       'pressao_atmosferica_min_na_hora_ant_aut_mb', 'radiacao_global_kjm2',
       'temperatura_do_ar__bulbo_seco_horaria_degc',
       'temperatura_do_ponto_de_orvalho_degc',
       'temperatura_maxima_na_hora_ant_aut_degc',
       'temperatura_minima_na_hora_ant_aut_degc',
       'temperatura_orvalho_max_na_hora_ant_aut_degc',
       'temperatura_orvalho_min_na_hora_ant_aut_degc',
       'umidade_rel_max_na_hora_ant_aut_', 'umidade_rel_min_na_hora_ant_aut_',
       'umidade_relativa_do_ar_horaria_', 'vento_direcao_horaria_gr_deg_gr',
       'vento_rajada_maxima_ms', 'vento_velocidade_horaria_ms']
    for col in colunas_numericas:
        if df[col].dtype == 'O':
            df[col] = df[col].str.replace(',','.')
        df[col] = pd.to_numeric(df[col])
    
    return df

In [2]:
df = pd.DataFrame()
for path in tqdm(os.listdir('../datasets/anos/2023')):
    df_temp = pd.read_csv(f'../datasets/anos/2023/{path}',header=8, sep=';',encoding_errors='ignore', encoding='latin-1', engine="pyarrow" )
    df_temp['path'] = path
    df = pd.concat([df, df_temp])

100%|██████████| 567/567 [03:14<00:00,  2.91it/s]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4966920 entries, 0 to 8759
Data columns (total 21 columns):
 #   Column                                                 Dtype  
---  ------                                                 -----  
 0   Data                                                   object 
 1   Hora UTC                                               object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       object 
 6   RADIACAO GLOBAL (Kj/m²)                                object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           object 
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)                   object 
 9   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)             object 
 10  TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)             object 
 11  TEMPER

In [5]:
df_trusted = transformar_dataframe(df.copy())

In [6]:
df_trusted[['source', 'seila','UF','cod_estacao','municipio','data_inicio','letra_seila','data_fim.csv']] = df_trusted['path'].str.split('_', expand=True)

In [7]:
df_trusted['data_fim'] = df_trusted['data_fim.csv'].str.replace('.CSV', '')

In [8]:
final_columns = ['municipio','data_inicio', 'data_fim', 'data_hora', 
       'UF', 'cod_estacao', 'letra_seila', 'precipitacao_total_horario_mm',
       'pressao_atmosferica_ao_nivel_da_estacao_horaria_mb',
       'pressao_atmosferica_maxna_hora_ant_aut_mb',
       'pressao_atmosferica_min_na_hora_ant_aut_mb', 'radiacao_global_kjm2',
       'temperatura_do_ar__bulbo_seco_horaria_degc',
       'temperatura_do_ponto_de_orvalho_degc',
       'temperatura_maxima_na_hora_ant_aut_degc',
       'temperatura_minima_na_hora_ant_aut_degc',
       'temperatura_orvalho_max_na_hora_ant_aut_degc',
       'temperatura_orvalho_min_na_hora_ant_aut_degc',
       'umidade_rel_max_na_hora_ant_aut_', 'umidade_rel_min_na_hora_ant_aut_',
       'umidade_relativa_do_ar_horaria_', 'vento_direcao_horaria_gr_deg_gr',
       'vento_rajada_maxima_ms', 'vento_velocidade_horaria_ms' ]

df_trusted = df_trusted[final_columns]
df_trusted.head(10)

,municipio,data_inicio,data_fim,data_hora,UF,cod_estacao,letra_seila,precipitacao_total_horario_mm,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb,pressao_atmosferica_maxna_hora_ant_aut_mb,...,temperatura_maxima_na_hora_ant_aut_degc,temperatura_minima_na_hora_ant_aut_degc,temperatura_orvalho_max_na_hora_ant_aut_degc,temperatura_orvalho_min_na_hora_ant_aut_degc,umidade_rel_max_na_hora_ant_aut_,umidade_rel_min_na_hora_ant_aut_,umidade_relativa_do_ar_horaria_,vento_direcao_horaria_gr_deg_gr,vento_rajada_maxima_ms,vento_velocidade_horaria_ms
0,BRASILIA,01-01-2023,31-12-2023,2023-01-01 00:00:00,DF,A001,A,0.0,887.7,887.7,...,20.9,20.0,19.2,17.8,91.0,87.0,87.0,187.0,3.3,1.2
1,BRASILIA,01-01-2023,31-12-2023,2023-01-01 01:00:00,DF,A001,A,0.0,888.1,888.1,...,20.1,19.2,17.8,17.4,90.0,87.0,90.0,153.0,2.9,0.8
2,BRASILIA,01-01-2023,31-12-2023,2023-01-01 02:00:00,DF,A001,A,0.0,887.8,888.1,...,19.5,19.0,17.8,17.3,90.0,89.0,90.0,145.0,2.5,1.5
3,BRASILIA,01-01-2023,31-12-2023,2023-01-01 03:00:00,DF,A001,A,0.0,887.8,887.9,...,19.4,19.1,17.8,17.5,91.0,90.0,91.0,162.0,3.2,1.4
4,BRASILIA,01-01-2023,31-12-2023,2023-01-01 04:00:00,DF,A001,A,0.0,887.6,887.9,...,19.7,19.1,18.1,17.4,91.0,90.0,90.0,140.0,5.7,2.7
5,BRASILIA,01-01-2023,31-12-2023,2023-01-01 05:00:00,DF,A001,A,0.0,886.7,887.6,...,19.7,19.1,18.1,17.7,92.0,90.0,92.0,128.0,7.1,2.0
6,BRASILIA,01-01-2023,31-12-2023,2023-01-01 06:00:00,DF,A001,A,0.0,886.3,886.8,...,19.1,18.4,17.7,17.1,92.0,92.0,92.0,196.0,4.0,0.4
7,BRASILIA,01-01-2023,31-12-2023,2023-01-01 07:00:00,DF,A001,A,0.0,886.4,886.5,...,18.6,18.4,17.4,17.1,93.0,92.0,93.0,98.0,2.5,0.8
8,BRASILIA,01-01-2023,31-12-2023,2023-01-01 08:00:00,DF,A001,A,0.0,886.5,886.6,...,18.7,18.2,17.3,16.8,93.0,91.0,92.0,51.0,3.4,0.8
9,BRASILIA,01-01-2023,31-12-2023,2023-01-01 09:00:00,DF,A001,A,0.0,887.1,887.1,...,18.3,18.0,17.0,16.8,93.0,92.0,93.0,121.0,3.5,1.6


In [10]:
df_trusted['municipio_partition'] = df_trusted['municipio'].apply(unidecode)\
                        .str.lower()\
                        .str.replace(' ', '_')\
                        .str.replace('[^a-z0-9_]', '', regex=True)

In [12]:
from deltalake.writer import write_deltalake

write_deltalake('../datasets/trusted_zone', df_trusted,
                description= 'Trusted zone from INMET data',
                partition_by= ['municipio_partition', 'data_inicio'])

In [1]:
from deltalake import DeltaTable

df_trusted_dt = DeltaTable('../datasets/trusted_zone').to_pandas()
df_trusted_dt.head(4)

,municipio,data_inicio,data_fim,data_hora,UF,cod_estacao,letra_seila,precipitacao_total_horario_mm,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb,pressao_atmosferica_maxna_hora_ant_aut_mb,...,temperatura_orvalho_max_na_hora_ant_aut_degc,temperatura_orvalho_min_na_hora_ant_aut_degc,umidade_rel_max_na_hora_ant_aut_,umidade_rel_min_na_hora_ant_aut_,umidade_relativa_do_ar_horaria_,vento_direcao_horaria_gr_deg_gr,vento_rajada_maxima_ms,vento_velocidade_horaria_ms,municipio_partition,__index_level_0__
0,ITUPORANGA,01-01-2023,31-12-2023,2023-11-15 14:00:00,SC,A863,A,0.0,955.5,956.2,...,18.8,18.2,92.0,87.0,88.0,353.0,2.8,0.1,ituporanga,7646
1,ITUPORANGA,01-01-2023,31-12-2023,2023-01-01 00:00:00,SC,A863,A,0.0,959.7,959.7,...,20.2,19.9,86.0,81.0,86.0,262.0,5.1,2.0,ituporanga,0
2,ITUPORANGA,01-01-2023,31-12-2023,2023-01-01 01:00:00,SC,A863,A,0.0,960.4,960.4,...,20.0,19.4,88.0,86.0,87.0,282.0,3.4,1.5,ituporanga,1
3,ITUPORANGA,01-01-2023,31-12-2023,2023-01-01 02:00:00,SC,A863,A,0.0,960.7,960.7,...,19.7,19.2,87.0,85.0,85.0,234.0,3.2,1.9,ituporanga,2


In [2]:
table_path = '../datasets/trusted_zone'

DeltaTable(table_path).to_pandas(
    partitions=[("municipio_partition", 'in', ["piracicaba"])]
)

,municipio,data_inicio,data_fim,data_hora,UF,cod_estacao,letra_seila,precipitacao_total_horario_mm,pressao_atmosferica_ao_nivel_da_estacao_horaria_mb,pressao_atmosferica_maxna_hora_ant_aut_mb,...,temperatura_orvalho_max_na_hora_ant_aut_degc,temperatura_orvalho_min_na_hora_ant_aut_degc,umidade_rel_max_na_hora_ant_aut_,umidade_rel_min_na_hora_ant_aut_,umidade_relativa_do_ar_horaria_,vento_direcao_horaria_gr_deg_gr,vento_rajada_maxima_ms,vento_velocidade_horaria_ms,municipio_partition,__index_level_0__
0,PIRACICABA,01-01-2023,31-12-2023,2023-11-02 05:00:00,SP,A726,A,0.0,947.9,948.8,...,15.3,15.2,81.0,79.0,81.0,157.0,4.8,2.9,piracicaba,7325
1,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 09:00:00,SP,A726,A,0.0,949.6,949.6,...,15.0,14.7,80.0,74.0,74.0,150.0,7.6,3.7,piracicaba,8745
2,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 17:00:00,SP,A726,A,0.0,947.8,948.8,...,18.4,15.4,49.0,40.0,42.0,177.0,5.7,2.1,piracicaba,8753
3,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 18:00:00,SP,A726,A,0.0,947.1,947.8,...,17.5,14.3,45.0,36.0,36.0,200.0,6.2,0.8,piracicaba,8754
4,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 20:00:00,SP,A726,A,0.0,946.6,946.8,...,14.7,12.8,40.0,32.0,39.0,181.0,7.0,2.6,piracicaba,8756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 05:00:00,SP,A726,A,0.0,949.2,949.8,...,16.4,16.2,81.0,78.0,80.0,160.0,7.2,3.6,piracicaba,8741
8756,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 06:00:00,SP,A726,A,0.0,949.1,949.4,...,16.2,15.8,81.0,78.0,81.0,174.0,5.9,2.3,piracicaba,8742
8757,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 07:00:00,SP,A726,A,0.0,949.1,949.2,...,16.4,15.3,84.0,80.0,80.0,182.0,5.3,1.7,piracicaba,8743
8758,PIRACICABA,01-01-2023,31-12-2023,2023-12-31 08:00:00,SP,A726,A,0.0,949.4,949.5,...,15.2,14.6,79.0,76.0,79.0,168.0,6.5,3.0,piracicaba,8744


In [3]:
print(DeltaTable(table_path).metadata())

Metadata(id: 64c48053-3072-4a87-897a-546bdd8b1333, name: None, description: Trusted zone from INMET data, partition_columns: ['municipio_partition', 'data_inicio'], created_time: 1730593073700, configuration: {})


In [4]:
DeltaTable(table_path).schema().to_pyarrow()

municipio: string
data_inicio: string
data_fim: string
data_hora: timestamp[us]
UF: string
cod_estacao: string
letra_seila: string
precipitacao_total_horario_mm: double
pressao_atmosferica_ao_nivel_da_estacao_horaria_mb: double
pressao_atmosferica_maxna_hora_ant_aut_mb: double
pressao_atmosferica_min_na_hora_ant_aut_mb: double
radiacao_global_kjm2: double
temperatura_do_ar__bulbo_seco_horaria_degc: double
temperatura_do_ponto_de_orvalho_degc: double
temperatura_maxima_na_hora_ant_aut_degc: double
temperatura_minima_na_hora_ant_aut_degc: double
temperatura_orvalho_max_na_hora_ant_aut_degc: double
temperatura_orvalho_min_na_hora_ant_aut_degc: double
umidade_rel_max_na_hora_ant_aut_: double
umidade_rel_min_na_hora_ant_aut_: double
umidade_relativa_do_ar_horaria_: double
vento_direcao_horaria_gr_deg_gr: double
vento_rajada_maxima_ms: double
vento_velocidade_horaria_ms: double
municipio_partition: string
__index_level_0__: int64

In [5]:
DeltaTable(table_path).history()

[{'timestamp': 1730593079739,
  'operation': 'WRITE',
  'operationParameters': {'partitionBy': '["municipio_partition","data_inicio"]',
   'mode': 'ErrorIfExists'},
  'clientVersion': 'delta-rs.0.21.0',
  'operationMetrics': {'execution_time_ms': 7034,
   'num_added_files': 565,
   'num_added_rows': 4966920,
   'num_partitions': 565,
   'num_removed_files': 0},
  'version': 0}]

In [6]:
DeltaTable(table_path).get_add_actions(flatten=True).to_pandas()

,path,size_bytes,modification_time,data_change,partition.municipio_partition,partition.data_inicio,num_records,null_count.municipio,min.municipio,max.municipio,...,max.vento_rajada_maxima_ms,null_count.vento_velocidade_horaria_ms,min.vento_velocidade_horaria_ms,max.vento_velocidade_horaria_ms,null_count.municipio_partition,min.municipio_partition,max.municipio_partition,null_count.__index_level_0__,min.__index_level_0__,max.__index_level_0__
0,municipio_partition=ituporanga/data_inicio=01-...,285378,2024-11-03 00:17:56.625,True,ituporanga,01-01-2023,8760,0,ITUPORANGA,ITUPORANGA,...,29.6,1509,-0.0,8.1,None,None,None,0,0,8759
1,municipio_partition=presidente_figueiredo/data...,198497,2024-11-03 00:17:56.626,True,presidente_figueiredo,01-01-2023,8760,0,PRESIDENTE FIGUEIREDO,PRESIDENTE FIGUEIREDO,...,11.7,5586,-0.0,4.6,None,None,None,0,0,8759
2,municipio_partition=rio_de_janeiromarambaia/da...,268808,2024-11-03 00:17:56.625,True,rio_de_janeiromarambaia,01-01-2023,8760,0,RIO DE JANEIRO-MARAMBAIA,RIO DE JANEIRO-MARAMBAIA,...,24.7,198,0.2,18.7,None,None,None,0,0,8759
3,municipio_partition=apui/data_inicio=01-01-202...,206268,2024-11-03 00:17:56.627,True,apui,01-01-2023,8760,0,APUI,APUI,...,15.2,5430,-0.0,5.2,None,None,None,0,0,8759
4,municipio_partition=bertioga/data_inicio=01-01...,279056,2024-11-03 00:17:56.625,True,bertioga,01-01-2023,8760,0,BERTIOGA,BERTIOGA,...,20.1,7299,0.2,3.9,None,None,None,0,0,8759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,municipio_partition=lages/data_inicio=01-01-20...,284068,2024-11-03 00:17:59.734,True,lages,01-01-2023,8760,0,LAGES,LAGES,...,24.8,4596,0.2,11.0,None,None,None,0,0,8759
561,municipio_partition=itapipoca/data_inicio=01-0...,296377,2024-11-03 00:17:59.706,True,itapipoca,01-01-2023,8760,0,ITAPIPOCA,ITAPIPOCA,...,17.9,33,0.1,9.8,None,None,None,0,0,8759
562,municipio_partition=teofilo_otoni/data_inicio=...,304256,2024-11-03 00:17:59.675,True,teofilo_otoni,01-01-2023,8760,0,TEOFILO OTONI,TEOFILO OTONI,...,20.8,0,-0.0,9.5,None,None,None,0,0,8759
563,municipio_partition=maria_da_fe/data_inicio=01...,291561,2024-11-03 00:17:59.687,True,maria_da_fe,01-01-2023,8760,0,MARIA DA FE,MARIA DA FE,...,17.6,0,-0.0,4.5,None,None,None,0,0,8759


In [45]:
database_path = "INMET_2023.db"
con = duckdb.connect(database_path)

In [46]:
con.commit()
con.execute(f"CREATE TABLE INMET_2023 AS SELECT * FROM df")

In [47]:
con.execute('SHOW TABLES').fetchall()

[('INMET_2023',)]

In [48]:
con.execute('SELECT * FROM INMET_2023').fetchdf()

,Data,Hora UTC,"PRECIPITAO TOTAL, HORRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m),"TEMPERATURA DO AR - BULBO SECO, HORARIA (C)",TEMPERATURA DO PONTO DE ORVALHO (C),TEMPERATURA MXIMA NA HORA ANT. (AUT) (C),TEMPERATURA MNIMA NA HORA ANT. (AUT) (C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREO HORARIA (gr) ( (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2023/01/01,0000 UTC,0,"887,7","887,7","887,2",None,"20,1","17,9","20,9",20,"19,2","17,8",91.0,87.0,87.0,187.0,"3,3","1,2",NaN
1,2023/01/01,0100 UTC,0,"888,1","888,1","887,7",None,"19,2","17,5","20,1","19,2","17,8","17,4",90.0,87.0,90.0,153.0,"2,9",",8",NaN
2,2023/01/01,0200 UTC,0,"887,8","888,1","887,8",None,"19,3","17,6","19,5",19,"17,8","17,3",90.0,89.0,90.0,145.0,"2,5","1,5",NaN
3,2023/01/01,0300 UTC,0,"887,8","887,9","887,7",None,"19,3","17,7","19,4","19,1","17,8","17,5",91.0,90.0,91.0,162.0,"3,2","1,4",NaN
4,2023/01/01,0400 UTC,0,"887,6","887,9","887,6",None,"19,7","18,1","19,7","19,1","18,1","17,4",91.0,90.0,90.0,140.0,"5,7","2,7",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4966915,2023/12/31,1900 UTC,0,"903,4","904,3","903,4","2834,4",26,"11,4","26,8","25,3","13,2","10,9",46.0,38.0,40.0,276.0,"6,3","1,9",NaN
4966916,2023/12/31,2000 UTC,0,"902,9","903,4","902,9","1088,7","24,6","12,6",26,"24,5",13,11,48.0,40.0,47.0,240.0,"4,3","1,4",NaN
4966917,2023/12/31,2100 UTC,0,"902,5","902,9","902,4","789,9","23,9","12,9","24,7","23,9","13,8","12,5",52.0,47.0,50.0,185.0,"6,6","4,6",NaN
4966918,2023/12/31,2200 UTC,0,"902,4","902,5","902,4","397,9","22,9","13,3",24,"22,9","13,4","12,7",55.0,50.0,55.0,178.0,"6,8","3,1",NaN
